# Building a CPI Bid Model (Part 2)
## Data Exploration
Previously, in Part 1 of this tutorial, we loaded Win and Conversion logs from S3 into our notebook and did some initial cleansing and aggregation of the data.  In Part 2, we'll do some exploration of our data and prepare it for the actual training of a model.

Let's start by reading in our dataframe from the last part and understanding how the features are distributed:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import boto3
import os
import seaborn as sns
from scipy import stats

df = pd.read_pickle("./data/step1.pkl")

# we don't really have any continuous features, so we'll conver most numeric fields to
for column in df.select_dtypes(include=['int64','float64', 'bool']).columns:
    if column in ['impressions', 'conversions']:
        df[column] = df[column].astype('int64')
        continue
    if column in ['lat_long_present']:
        df[column] = df[column].astype('int64')
    df[column] = df[column].astype('object')

# Frequency tables for each categorical feature
for column in df.select_dtypes(include=['object']).columns:
    display(pd.crosstab(index=df[column], columns='% observations', normalize='columns'))


# Histograms for each numeric features
display(df.describe())
%matplotlib inline
hist = df.hist(bins=30, sharey=True, figsize=(10, 10))

A couple of observations from this analysis:
* it's a little difficult to tell much from the conversion and impression counts since we dont really have a metric that tells us how well a row performs.  We'll fix this by adding a "conversion rate" column.
* `ad_position` is almost always "above the fold
* `content_coppa_flag` is empty 99% of the time and when its not empty, the value is always 0.  Empty and 0 have the same meaning here.  We'll drop this field since it won't give us much signal.
* `video_start_delay` is similar; the value is either 0 or not present which implies 0 so we'll drop that one as well.

Let's make these changes:

In [ ]:
# add "conversion_rate"
df['conversion_rate'] = df['conversions']/df['impressions']

# drop unwanted columns
df = df.drop(['content_coppa_flag', 'video_start_delay', 'ad_position'], axis=1)

Next, let's look at how our features relate to the `conversion_rate` itself.  We'll bin the conversion rate to make the data easier to view,a nd then we'll look for interesting distributions that show us a particular attribute might be a good predictor of high conversion:

In [ ]:
df['conversion_rate_bin'] = pd.cut(df['conversion_rate'], 10)
for column in df.select_dtypes(include=['object']).columns:
        display(pd.crosstab(index=df[column], columns=df['conversion_rate_bin'], normalize='columns'))

Notice that:
* most of the `app_bundle`s exhibit pretty poor conversion rates, but there are a small number that contribute to very good conversion rates.  We'll want to keep this in the model as it will likely give us a strong signal
* ad format features (e.g. `banner_height` and `banner_width`) provide a lot of signal.  We'll keep these in as well.
* a few specific device makes (Apple and Samsung) provide most of the signal.  Device model also provides some signal but its not as dramatic.
* `display_manager` is interesting: applovin accounts for a significant portion of the dataset but with apploving the distribution is pretty flat. On the otherhand mopub accounts for a large portion of the high conversion rate lines

In [ ]:
df = df.drop(['placement', 'platform_device_model', 'display_manager_ver'], axis=1)

## Transformation
Now that we understand our dataset a little better, we can finish preparing it for model training.  This can include several steps:
* Removing rows that have very few impressions to prevent them from skewing the model
* Handle missing values: right now, all our missing values are encoded as "-1".  If we had numerical features, we might employ some technique to impute the values, but since all our features are categorical and "null" is actually a valid option on a real bid request, we will just leave the "-1" values and reformat them at the very end.
* Converting categorical to numeric: In order for the model to understand our categorical features we need to encode them to numeric values.  We'll do this with [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html).
* Handling odd data distributions: many models work best with evenly distributed data, but since we don't really have numeric/continuous features we'll skip this for now.
* Dropping unneeded columns: since we added a column for `conversion_rate`, we'll use this as the value we are trying to predict and drop `impressions` and `conversions`
* Splitting the data: To avoid overfitting issues, we'll split the dataset into 3 groups: 70% for training, 20% for testing, 10% for validating


Let's get started by removing low-impression rows:

In [ ]:
df = df.loc[df['impressions'] > 5]
print(df.shape)
df.head()

Next, let's look at the distribution of conversion data after having removed the "low count" rows:

In [ ]:
# Histograms for each numeric features
display(df.describe())
%matplotlib inline
fig, ax = plt.subplots()
df['conversion_rate'].hist(ax=ax, bins=100, bottom=0.1)
ax.set_yscale('log')

We can see that for the most part, we have very low conversion scores (usually 0), but in some cases there are very high conversion scores.  These scores are probably outliers since in the real world its very unlikely that a conversion rate would be ~100% for any combination of inventory.  Let's remove rows with out-lying conversion data.  We'll do this using z-scores to determine the number of standard deviations from the mean and then remove anything that has a z-score greater than 3.

In [ ]:
df['z'] = np.abs(stats.zscore(df['conversion_rate']))
df = df.loc[df['z'] < 3.0]
df = df.drop(['z'], axis=1)

display(df.describe())
%matplotlib inline
fig, ax = plt.subplots()
df['conversion_rate'].hist(ax=ax, bins=100, bottom=0.1)
ax.set_yscale('log')

Now that we've removed the outliers, we'll need to encode all our categorical variables so the model can understand them.

In [ ]:
model_df = pd.get_dummies(df.to_sparse(), sparse=True, prefix_sep='-')
model_df.head()

Now let's drop the unneeded cols

In [ ]:
model_df = model_df.drop(['conversions', 'impressions'], axis=1)
for col in model_df:
    if col.startswith('conversion_rate_bin'):
        model_df = model_df.drop([col], axis=1)

model_df.head()

In [ ]:
model_df = model_df.loc[:, ~model_df.columns.duplicated()]  # make sure we don't have any duplicated columns after the encoding
print(model_df.shape)
model_df.head()

Finally, split the data into our 3 cuts:

In [ ]:
train_data = model_df.sample(frac=.7).to_dense()
validation_data = model_df.drop(train_data.index).sample(frac=.66).to_dense()
test_data = model_df.drop(train_data.index).drop(validation_data.index).to_dense()

We're now ready to export our data for model training.  To do this, we need to get it into the format SageMaker expects: dependent variable in the first column, no headers, csv format.

In [ ]:
pd.concat([train_data['conversion_rate'], train_data.drop(['conversion_rate'], axis=1)], axis=1).to_dense().to_csv('data/train.csv', index=False, header=False)
pd.concat([validation_data['conversion_rate'], validation_data.drop(['conversion_rate'], axis=1)], axis=1).to_dense().to_csv('data/validation.csv', index=False, header=False)

Since we're going to use SageMaker to train our model, we'll need to write the data to S3.  Let's do that now:

In [ ]:
bucket = 'beeswax-tmp-us-east-1'
prefix = 'bid-models-test-data/canary/sagemaker'
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('data/train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('data/validation.csv')

# drop the data files from disk, they are huge and we don't want to keep them
os.remove('data/train.csv')
os.remove('data/validation.csv')

To wrap up this part of the tutorial, we'll save our state so we can re-use it again in the next part:

In [ ]:
df.to_pickle('data/step2.pkl')
model_df.to_pickle('data/step2-model.pkl')
test_data.to_pickle('data/step2-test.pkl')
train_data.to_pickle('data/step2-train.pkl')

That's it! In the next part of this tutorial, we'll get to the fun part: training an actual model.